In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [24]:
import cv2
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

import shutil
import os, sys, glob

GradeCode = {
    '3+3': 0,
    '4+4': 1,
    '4+5': 2,
    'BN':  3,
    'ST':  4
}

GradeCodeInv = {
    0: '3+3',
    1: '4+4',
    2: '4+5',
    3: 'BN',
    4: 'ST'
}

# Given the annotation code, what labels are we looking for?
GradeTargets = {
    0: 0,
    1: 1,
    2: 1,
    3: 2,
    4: 3
}

epithelium_labels = [0,1,2]
proposal_labels = {
    0: 'Low Grade',
    1: 'High Grade',
    2: 'Benign',
    3: 'Stroma'
}

# prob_list = sorted(glob.glob('/Users/nathaning/Dropbox/projects/semantic_pca/figures/probs/*probability.jpg'))
# prob_list = sorted(glob.glob('/Users/nathaning/Dropbox/projects/semantic_pca/wsi_results/segnet_basic_rawprob/*RGB.png'))
# anno_list = sorted(glob.glob('/Users/nathaning/Dropbox/projects/semantic_pca/data/annotations/wsi_annotation/*.png'))

prob_list = sorted(glob.glob('/home/nathan/histo-seg/semantic-pca/analysis_wsi/segnet_basic/*npy'))
anno_list = sorted(glob.glob('/home/nathan/Dropbox/projects/semantic_pca/data/annotations/wsi_annotation/*.png'))

# prob_base = [os.path.basename(os.path.splitext(pb)[0]).replace('_probability', '') for pb in prob_list]
prob_base = [os.path.basename(os.path.splitext(pb)[0]).replace('_probability', '') for pb in prob_list]
anno_base = [os.path.basename(os.path.splitext(ab)[0]) for ab in anno_list]

prob_list = [p_path for pb, p_path in zip(prob_base, prob_list) if pb in anno_base]
# prob_base = [os.path.basename(os.path.splitext(pb)[0]).replace('_probability', '') for pb in prob_list]
prob_base = [os.path.basename(os.path.splitext(pb)[0]).replace('_probability', '') for pb in prob_list]
anno_list = [a_path for ab, a_path in zip(anno_base, anno_list) if ab in prob_base]

print len(prob_list), len(anno_list)

30 30


In [38]:
def class_masks(prob, mode='Argmax', thr=0.5):
    n_class = prob.shape[-1]
    if mode=='Argmax':
        prob = np.argmax(prob, axis=2)
        
    masks = [prob == n for n in range(n_class)]
    
    return masks

def eval_region(proposed, true_grade, verbose=True):
    area = np.sum(proposed)
        
    epith_area = 0.0
    st_area = proposed[-1]
    for k in epithelium_labels:
        epith_area += proposed[k]
        
    TP = proposed[GradeTargets[true_grade]]    
    if true_grade == 4:
        TP_pct = 100*TP/st_area
    else:
        TP_pct = 100*TP/epith_area
#     FN = epith_area - TP
#     FN_pct = 100*FN/epith_area

    if verbose:
        print 'Region graded as {} is {} total area'.format(GradeCodeInv[true_grade], area)
        for k in proposal_labels.iterkeys():
            print '\t{}: {}'.format(proposal_labels[k], proposed[k])
        print '\tEpithelium area: {}'.format(epith_area)
        print '\tTrue postive: {:3.3f}%'.format(TP_pct)
    
    return TP_pct
#     print '\tFalse negative: {:3.3f}%'.format(FN_pct)

In [41]:
grade_results = {
    0: [],
    1: [],
    2: [],
    3: [],
    4: [],
}

for ppath, apath in zip(prob_list, anno_list):
#     ppath = prob_list[2]
#     apath = anno_list[2]

    print os.path.basename(ppath)
    # pimg = cv2.imread(ppath, -1)
    pimg = np.load(ppath)
    aimg = cv2.imread(apath, -1)

    # print pimg.shape, pimg.min(), pimg.max()
    # print aimg.shape, aimg.min(), aimg.max(), np.unique(aimg)

    pmax = np.argmax(pimg, axis=2)
    # plt.matshow(pmax); plt.colorbar()

    masks = class_masks(pimg)
    # for mask in masks:
    #     plt.matshow(mask); plt.colorbar()

    annotations, _ = cv2.findContours((aimg<255).astype(np.uint8),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    for anno in annotations:
    #     anno = annotations[1]
        anno_mask = np.zeros_like(aimg)
        cv2.drawContours(anno_mask, [anno], 0, (1), -1)
        anno_mask = anno_mask==1
        grade = np.unique(aimg[anno_mask])[0]
        # print grade
        # plt.matshow(anno_mask); plt.colorbar()

        # Check the content
        proposed = [(mask[anno_mask]).sum() for mask in masks]
        TP = eval_region(proposed, grade, verbose=True)

        grade_results[grade].append(TP)

for key in grade_results:
    print GradeCodeInv[key], ':\t', np.mean(grade_results[key]), '+/-', np.std(grade_results[key])

1304700_probability.npy
Region graded as 3+3 is 149788 total area
	Low Grade: 7945
	High Grade: 0
	Benign: 0
	Stroma: 141843
	Epithelium area: 7945.0
	True postive: 100.000%
Region graded as BN is 146527 total area
	Low Grade: 142
	High Grade: 0
	Benign: 688
	Stroma: 145697
	Epithelium area: 830.0
	True postive: 82.892%
1304701_probability.npy
Region graded as 3+3 is 44500 total area
	Low Grade: 355
	High Grade: 0
	Benign: 0
	Stroma: 44145
	Epithelium area: 355.0
	True postive: 100.000%
Region graded as BN is 638192 total area
	Low Grade: 975
	High Grade: 0
	Benign: 5693
	Stroma: 631524
	Epithelium area: 6668.0
	True postive: 85.378%
Region graded as ST is 188696 total area
	Low Grade: 52
	High Grade: 0
	Benign: 0
	Stroma: 188644
	Epithelium area: 52.0
	True postive: 100.000%
1304702_probability.npy
Region graded as BN is 1011972 total area
	Low Grade: 8021
	High Grade: 3328
	Benign: 258320
	Stroma: 742303
	Epithelium area: 269669.0
	True postive: 95.792%
Region graded as 3+3 is 143097

In [17]:
len(annotations)

2